<a href="https://colab.research.google.com/github/jiyeonjin/0624_new/blob/main/0717_yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# YOLOv8 영상 객체 탐지 코드 (ImageNet 분류기에서 확장)

from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
from google.colab import files
import os

# YOLOv8 모델 로딩 (YOLOv8s 버전 사용)
model = YOLO('yolov8s.pt')  # 필요에 따라 yolov8n.pt, yolov8m.pt 등으로 교체 가능

# 영상 경로 설정 (Colab에 업로드)
print("\n📁 분석할 영상을 업로드해주세요...")
uploaded = files.upload()
if not uploaded:
    print("❌ 영상이 업로드되지 않았습니다!")
    exit()

video_path = list(uploaded.keys())[0]
print(f"🎬 업로드된 영상: {video_path}")

# 비디오 캡처
cap = cv2.VideoCapture(video_path)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

# 저장용 비디오 설정
output_path = 'output_yolo.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0

print("\n🚀 YOLOv8 객체 인식 시작...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    print(f"\r📷 프레임 {frame_count} 처리 중...", end='')

    # YOLOv8 추론
    results = model(frame, verbose=False)[0]

    # 결과 시각화
    annotated_frame = results.plot()  # 바운딩 박스 시각화

    # 출력 저장
    out.write(annotated_frame)

cap.release()
out.release()

print("\n✅ 영상 분석 완료!")
print(f"💾 결과 저장: {output_path}")

# Colab에서 다운로드 링크 제공
files.download('/content/output_yolo.mp4')


In [ ]:
# 📦 YOLOv8 설치 및 라이브러리 임포트
!pip install ultralytics --quiet

import cv2
import numpy as np
import os
import uuid
from ultralytics import YOLO
from google.colab import files
from IPython.display import Video

# 📁 파일 업로드
print("📁 파일을 업로드하세요...")
uploaded = files.upload()

# 업로드된 파일 확인
if not uploaded:
    print("❌ 업로드된 파일이 없습니다!")
    exit()

# 업로드한 첫 번째 파일 경로
video_path = next(iter(uploaded))
print(f"📹 업로드된 파일: {video_path}")

# 파일 존재 확인
if not os.path.exists(video_path):
    print(f"❌ 파일을 찾을 수 없습니다: {video_path}")
    exit()

try:
    # YOLOv8 모델 로드 (YOLOv8n은 가장 가벼운 모델)
    print("🤖 YOLOv8 모델 로딩 중...")
    model = YOLO('yolov8n.pt')
    print("✅ 모델 로드 완료!")

    # 🎞️ 영상 열기
    print("🎞️ 영상 파일 열기 중...")
    cap = cv2.VideoCapture(video_path)

    # 영상이 제대로 열렸는지 확인
    if not cap.isOpened():
        print("❌ 영상 파일을 열 수 없습니다!")
        print("지원되는 형식: mp4, avi, mov, mkv 등")
        exit()

    # 영상 정보 가져오기
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"📊 영상 정보:")
    print(f"   - 해상도: {width}x{height}")
    print(f"   - FPS: {fps}")
    print(f"   - 총 프레임: {total_frames}")

    # FPS가 0이거나 비정상적인 경우 기본값 설정
    if fps <= 0 or fps > 120:
        fps = 30.0
        print(f"⚠️  FPS를 기본값 {fps}로 설정했습니다.")

    # 🔄 출력 영상 저장 준비
    output_path = f"output_{uuid.uuid4().hex[:8]}.mp4"

    # 다양한 코덱 시도
    codecs_to_try = ['mp4v', 'XVID', 'MJPG', 'H264']
    out = None

    for codec in codecs_to_try:
        fourcc = cv2.VideoWriter_fourcc(*codec)
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        if out.isOpened():
            print(f"✅ 비디오 작성기 초기화 성공 (코덱: {codec})")
            break
        else:
            out.release()

    if out is None or not out.isOpened():
        print("❌ 비디오 작성기를 초기화할 수 없습니다!")
        cap.release()
        exit()

    frame_count = 0
    processed_frames = 0

    print("🚀 프레임 처리 시작...")

    # 📦 프레임 처리
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # 진행률 표시
        if total_frames > 0:
            progress = (frame_count / total_frames) * 100
            print(f"\r📷 프레임 {frame_count}/{total_frames} 처리 중... ({progress:.1f}%)", end='')
        else:
            print(f"\r📷 프레임 {frame_count} 처리 중...", end='')

        try:
            # YOLOv8 추론
            results = model(frame, verbose=False)[0]

            # 결과 시각화
            annotated_frame = results.plot()

            # RGB → BGR로 변환하여 저장
            if annotated_frame.dtype != np.uint8:
                annotated_frame = annotated_frame.astype(np.uint8)

            bgr_frame = cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR)

            # 프레임 크기 확인 및 조정
            if bgr_frame.shape[:2] != (height, width):
                bgr_frame = cv2.resize(bgr_frame, (width, height))

            out.write(bgr_frame)
            processed_frames += 1

        except Exception as e:
            print(f"\n⚠️  프레임 {frame_count} 처리 중 오류: {str(e)}")
            continue

    # 종료 처리
    cap.release()
    out.release()

    print(f"\n✅ 영상 처리 완료!")
    print(f"   - 처리된 프레임: {processed_frames}/{frame_count}")
    print(f"   - 저장 경로: {output_path}")

    # 출력 파일 존재 확인
    if os.path.exists(output_path) and os.path.getsize(output_path) > 0:
        print("📺 처리된 영상을 재생합니다...")
        # Colab에서 영상 바로 보기
        display(Video(output_path, embed=True))

        # 파일 다운로드 옵션 제공
        print("📥 처리된 영상을 다운로드하려면 아래 코드를 실행하세요:")
        print(f"files.download('{output_path}')")
    else:
        print("❌ 출력 파일이 생성되지 않았습니다.")

except Exception as e:
    print(f"❌ 오류가 발생했습니다: {str(e)}")
    print("📝 문제 해결 팁:")
    print("1. 영상 파일이 손상되지 않았는지 확인하세요")
    print("2. 지원되는 형식인지 확인하세요 (mp4, avi, mov 등)")
    print("3. 파일 크기가 너무 크지 않은지 확인하세요")
    print("4. 런타임을 다시 시작해보세요")

    # 정리 작업
    try:
        cap.release()
        if 'out' in locals():
            out.release()
    except:
        pass
files.download('output_81924767.mp4')

유튜브 링크 업로드 하는 코드

In [ ]:
!pip install ultralytics yt-dlp --quiet

import cv2
import uuid
import yt_dlp
from ultralytics import YOLO
from IPython.display import Video

# YouTube 링크 입력
youtube_url = input("YouTube 링크를 입력하세요: ")

# YouTube 영상 다운로드
ydl_opts = {
    'format': 'best[height<=720]',
    'outtmpl': 'youtube_video.%(ext)s',
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([youtube_url])

# 다운로드된 파일 경로 찾기
import os
video_files = [f for f in os.listdir('.') if f.startswith('youtube_video.')]
video_path = video_files[0] if video_files else None

# 모델 로드 및 영상 열기
model = YOLO('yolov8n.pt')
cap = cv2.VideoCapture(video_path)

# 영상 정보
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) if cap.get(cv2.CAP_PROP_FPS) > 0 else 30.0

# 출력 비디오 설정
output_path = f"output_{uuid.uuid4().hex[:8]}.mp4"
fourcc = cv2.VideoWriter_fourcc(*'H264')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 비디오 writer 확인
if not out.isOpened():
    print("H264 실패, XVID 시도 중...")
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    output_path = f"output_{uuid.uuid4().hex[:8]}.avi"
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 프레임 처리
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    print(f"\r프레임 {frame_count} 처리 중...", end='')

    # YOLO 추론 및 시각화
    results = model(frame, verbose=False)[0]
    annotated_frame = results.plot()
    bgr_frame = cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR)
    out.write(bgr_frame)

# 종료 및 결과 출력
cap.release()
out.release()
print(f"\n✅ 완료! 총 {frame_count}프레임 처리")
print(f"📁 출력 파일: {output_path}")

# 파일 크기 확인
import os
file_size = os.path.getsize(output_path) / (1024*1024)
print(f"📊 파일 크기: {file_size:.2f}MB")

# 결과 영상 재생 (여러 방법 시도)
try:
    from IPython.display import HTML
    print("🎬 영상 재생 시도 중...")

    # HTML5 video 태그로 재생
    video_html = f"""
    <video width="640" height="480" controls>
        <source src="{output_path}" type="video/mp4">
        <source src="{output_path}" type="video/avi">
        Your browser does not support the video tag.
    </video>
    """
    display(HTML(video_html))

    # 다운로드 링크도 제공
    print("\n📥 다운로드 링크:")
    from google.colab import files
    files.download(output_path)

except Exception as e:
    print(f"재생 오류: {e}")
    print("대신 파일 다운로드를 시도합니다...")
    files.download(output_path)